In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime
np.random.seed(1)

In [2]:
ks_2019 = pd.read_csv('./kickstarter_2019.csv')

In [13]:
ks_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44158 entries, 0 to 44157
Data columns (total 50 columns):
backers_count               44158 non-null int64
blurb                       44158 non-null object
category                    44158 non-null object
category_name               44158 non-null object
category_specific           44158 non-null object
converted_pledged_amount    44158 non-null int64
country                     44158 non-null object
country_displayable_name    44158 non-null object
created_at                  44158 non-null int64
date_created                44158 non-null object
creator                     44158 non-null object
currency                    44158 non-null object
currency_symbol             44158 non-null object
currency_trailing_code      44158 non-null bool
current_currency            44158 non-null object
deadline                    44158 non-null int64
date_deadline               44158 non-null object
disable_communication       44158 non-null bool

In [3]:
ks_2019.shape

(44158, 50)

In [16]:
ks_2019['currency'].value_counts()

USD    26087
GBP     6101
EUR     4703
CAD     2351
AUD     1273
MXN     1038
HKD      747
SEK      433
SGD      339
JPY      291
NZD      235
DKK      231
CHF      222
NOK      107
Name: currency, dtype: int64

In [31]:
ks_2019['url_project'][0]

'https://www.kickstarter.com/projects/469036700/louli-a-lecole?ref=discovery_category_newest'

In [34]:
ks_2019[['currency','current_currency','goal','pledged','converted_pledged_amount','url_project']].head()

,currency,current_currency,goal,pledged,converted_pledged_amount,url_project
0,EUR,USD,600.0,631.0,709,https://www.kickstarter.com/projects/469036700...
1,USD,USD,30000.0,82.0,82,https://www.kickstarter.com/projects/gplive/gp...
2,GBP,USD,782.0,889.0,1153,https://www.kickstarter.com/projects/281093153...
3,USD,USD,7000.0,7415.0,7415,https://www.kickstarter.com/projects/genesisfi...
4,USD,USD,10000.0,1.0,1,https://www.kickstarter.com/projects/recoverym...


In [15]:
ks_2019['current_currency'].value_counts()

USD    44014
CAD       48
GBP       48
AUD       36
EUR       12
Name: current_currency, dtype: int64

In [12]:
ks_2019['currency_symbol'].value_counts()

$      32070
£       6101
€       4703
kr       771
¥        291
Fr       222
Name: currency_symbol, dtype: int64

In [18]:
ks_2019['converted_pledged_amount'].head()

0     709
1      82
2    1153
3    7415
4       1
Name: converted_pledged_amount, dtype: int64

In [4]:
ks_2019['state'].value_counts()

successful    30002
failed         7201
live           6063
canceled        845
suspended        47
Name: state, dtype: int64

In [5]:
success_failed_ks_2019 = ks_2019.loc[ks_2019['state'].isin(['successful','failed'])] #성공, 실패 프로젝트 데이터들만 가져오기
success_failed_ks_2019.shape

(37203, 50)

In [6]:
success_failed_ks_2019['url_project'].head()

0    https://www.kickstarter.com/projects/469036700...
1    https://www.kickstarter.com/projects/gplive/gp...
2    https://www.kickstarter.com/projects/281093153...
3    https://www.kickstarter.com/projects/genesisfi...
5    https://www.kickstarter.com/projects/gtsa/free...
Name: url_project, dtype: object

In [7]:
success_failed_url = success_failed_ks_2019['url_project']

In [8]:
success_failed_url[0]

'https://www.kickstarter.com/projects/469036700/louli-a-lecole?ref=discovery_category_newest'

In [9]:
type(success_failed_url)

pandas.core.series.Series

In [21]:
failed_url_df = pd.read_csv('./failed_project_url_2019.csv', header=None, names=['url'])

In [22]:
failed_url = failed_url_df['url']

In [23]:
print(type(failed_url))
failed_url.head()

<class 'pandas.core.series.Series'>


0    https://www.kickstarter.com/projects/gplive/gp...
1    https://www.kickstarter.com/projects/gtsa/free...
2    https://www.kickstarter.com/projects/2rate/2ra...
3    https://www.kickstarter.com/projects/945795009...
4    https://www.kickstarter.com/projects/sloproces...
Name: url, dtype: object

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import random
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [27]:
#test
path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver'
test_driver = webdriver.Chrome(path)
test_driver.get("https://www.naver.com")
time.sleep(5)
test_driver.quit()

In [30]:
#크롤링을 시작하는 것일 때 사용

options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver'
driver = Chrome(executable_path=path, options=options)
content_list = []

for idx, url in enumerate(url_list):
    driver.get(url)
    time.sleep(random.uniform(1,10))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content) #한 프로젝트의 페이지 내 모든 설명/내용들을 하나로 모으고
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #np.nan
        
        #설명/내용 모은 것을 risk_challenge 모은 것과 index번호와 같이 종합적인 리스트인 content_list에 저장
        content_list.append([idx, contents_collected, risk_challenge])
        df_content = pd.DataFrame(content_list, columns=['index','content','risk_challenge']) #종합 리스트를 dataframe으로 만들고
        df_content.to_csv('failed_ks_content_crawled.csv', index=False, mode='w', encoding='UTF-8') #csv파일로 저장
        print(idx) #한 url에 대한 크롤링이 끝날 때마다 해당 index번호 출력
        
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex) #오류 내용 출력
        
        error_list = [] #오류난 곳의 index와 url을 담을 리스트
        error_list.append([idx, url]) #오류가 난다면 해당 오류의 index와 url을 리스트로 묶어, dataframe변환 및 csv파일 저장
        pd.DataFrame(error_list, columns=['index','error_url']).to_csv('failed_ks_middle_errors.csv', index=False, mode='w', encoding='UTF-8')
        break #크롤링 중단
    
    #time.sleep(1)
    
driver.close()

#만약 에러가 났을 시, 아래의 코드로 실행

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
Unexpected error at 29 'NoneType' object has no attribute 'select'


In [195]:
from bs4 import BeautifulSoup
import requests
import time
import random
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [33]:
#오류 난 곳부터 크롤링은 이것 사용

#content_list = [] : 오류가 나기 전까지의 크롤링 된 내용들은 여기에 이미 저장되어 있다. 이걸 시행하게 되면 이 리스트가 초기화된다;
#하지만 우린 데이터를 이어서 append할 것이기에, 이 실행문에선 content_list = [] 빼고 아래에서 실행 돌리면서 이어서 append 할 것이다

options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

#content_list = []

for idx, url in enumerate(url_list[200:], start=200): #오류나면 오류난 부분 index 번호를 [ :]와 start= 에 넣으세요
    driver.get(url)
    time.sleep(random.uniform(1,10))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #np.nan
        
        #오류난 곳 부터 다시 크롤링 할 때,
        #오류 나기 전까지의 데이터들이 다 저장되어 있는 종합적인 리스트인 content_list 변수에는 계속 이어서 크롤링 되는 것 append하고
        #content_list에 모으는 것 유지할 수 있도록
        content_list.append([idx, contents_collected, risk_challenge])
        
        #매 페이지 긁어온 것마다 바로바로 content csv파일에 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled.csv', index=False, header=False, mode='a', encoding='UTF-8')
        #with open('ks_content_crawled.csv', 'a', encoding='utf-8') as fc:
        #    df_content.to_csv(fc, index=False, header=False, encoding='UTF-8')
        print(idx)
        
    except Exception as ex:
        #오류가 난다면, 일단 무슨 오류가 어디서 났는지 알리고
        print("Unexpected error at {}".format(idx), ex)
        
        #오류가 나면 크롤링을 중지시킬 것이기에, index와 url을 담은 csv 파일을 매번 새로 만들것이다 (append가 아닌 덮어쓰기 처럼 된다)
        error_list = [] #오류가 난 곳부터 다시 크롤링을 하는 것이기에, 새로 뜨는 오류를 담을 리스트 새로 생성
        error_list.append([idx, url]) #그리고 리스트로 묶어 오류를 저장할 새 csv파일 생성 및 저장
        pd.DataFrame(error_list, columns=['index','error_url']).to_csv('ks_middle_errors.csv', index=False, mode='w', encoding='UTF-8')
        #with open('ks_middle_errors.csv', 'a', encoding='utf-8') as fe:
        #    df_content.to_csv(fe, index=False, encoding='UTF-8')
        break
    
    #time.sleep(1)
    
driver.close()

132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: headless chrome=79.0.3945.117)


In [ ]:
#크롤링이 계속(continue)되도록 돌릴 때 사용

options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

#content_list = []

for idx, url in enumerate(url_list):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #결측치 처리 전엔 --> np.nan 
        
        #오류 나기 전까지 저장되어 있는 content_list에는 이어서 append하고; 그래서 content_list에 모으는 것도 유지할 수 있도록
        content_list.append([idx, contents_collected, risk_challenge])
        
        #오류 난 후부턴 다시 크롤링 할 때, 매 페이지 긁어온 것마다 바로바로 csv파일에 한줄씩 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled.csv', index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)
        
    except Exception as ex:
        #오류가 난다면, 일단 무슨 오류가 어디서 났는지 알리고
        print("Unexpected error at {}".format(idx), ex)
        
        #오류 부분의 index와 url은 오류만 모으는 csv파일에 계속 append 되도록 하고
        error_list = []
        error_list.append([idx, url])
        pd.DataFrame(error_list, columns=['index','error_url']).to_csv('ks_middle_errors.csv', index=False, header=False, mode='a', encoding='UTF-8')
        
        #따로 계속 모으던 content_list 변수엔 index, url, nan/null값을 대신 넣을거다
        content_list.append([idx, url, np.nan]) #contents_collected 자리엔 오류난 곳 url을, risk_challenge자리엔 nan/null값을 대신 삽입
        
        #content모으는 csv파일에도 에러난 곳은 index, url, nan/null값이 대신 들어가게 하자
        error_into_saved_csv = []
        error_into_saved_csv.append([idx, url, np.nan]) #오류난 곳의 index와 url 그리고 null값을 묶어서 리스트에 전달하고, 이어서 content csv파일에 append 및 저장
        pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv('failed_ks_content_crawled.csv', index=False, header=False, mode='a', encoding='UTF-8')
        
        continue
    
    #time.sleep(1)
    
driver.close()

In [197]:
risk = np.nan
risk

nan

In [198]:
test = pd.read_csv('test.csv', header=None, names=['url'])
test_url = test['url']
test_url[1]

'https://www.kickstarter.com/projects/gtsa/free-global-transport-sharing-app?ref=discovery_category_newest'

In [ ]:
#크롤링 block 당했거나 오류가 났을 때, continue하고 다음 url로 바로 넘어가는게 아니라 약 2분 정도 간격으로 10번까지 찔러보다가 되면 scrape하고 안 되면 continue되도록 한다

options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

content_list = []

for idx, url in enumerate(test):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        
        #오류 나기 전까지 저장되어 있는 content_list에는 이어서 append하고; 그래서 content_list에 모으는 것도 유지할 수 있도록
        content_list.append([idx, contents_collected, risk_challenge])
        
        #오류 난 후부턴 다시 크롤링 할 때, 매 페이지 긁어온 것마다 바로바로 csv파일에 한줄씩 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv('failed_ks_content_crawled.csv', index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)
        
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 11:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list.append([idx, contents_collected, risk_challenge])

                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled.csv', index=False, header=False, mode='a', encoding='UTF-8')
                print(idx)
                break
                
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        #만약 재시도를 10번 했는데도 크롤링에 실패한다면, 원래 했던대로 오류 index, url, nan값을 csv파일들에 저장
        if count == 11:
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv('ks_middle_errors.csv', index=False, header=False, mode='a', encoding='UTF-8')
        
            content_list.append([idx, error_url, np.nan])
        
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, error_url, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled.csv', index=False, header=False, mode='a', encoding='UTF-8')
        
        continue
    
driver.close()